In [ ]:
# importando uma das principais bibliotecas de tratamento de texto
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

# importando o modelo que vamos usar para classificar a frase
from textblob import TextBlob

import pandas as pd

##  Coleta de dados

In [11]:
frase = 'Gostei muito do filme, mas achei o final muito ruim'
exemplo = TextBlob(frase)
exemplo = exemplo.translate(from_lang='pt', to='en')

## Pré processamento

In [12]:
exemplo.sentences

[Sentence("I really liked the movie, but I found the ending very bad")]

In [13]:
exemplo.words

WordList(['I', 'really', 'liked', 'the', 'movie', 'but', 'I', 'found', 'the', 'ending', 'very', 'bad'])

In [14]:
exemplo.noun_phrases  # frases nominais

WordList([])

In [17]:
exemplo.sentiment  # polaridade e subjetividade

# polaridade: -1 a 1. Número expressa o quão positivo ou negativo é o texto
# subjetividade: 0 a 1. Número expressa o quão subjetivo é o texto

Sentiment(polarity=-0.15499999999999992, subjectivity=0.8333333333333334)

In [24]:
text_tokens = word_tokenize(str(frase))

# Mostra somente as palavras que sao stopwords
tokens_sem_sw = [
    word for word in text_tokens if word not in stopwords.words("portuguese")
]

tokens_sem_sw

['Gostei', 'filme', ',', 'achei', 'final', 'ruim']

In [25]:
filtered_sentence = (" ").join(tokens_sem_sw)
print(filtered_sentence)

Gostei filme , achei final ruim


In [29]:
exemplo = TextBlob(filtered_sentence)
exemplo = exemplo.translate(from_lang='pt', to='en')

In [35]:
polaridade = exemplo.sentiment.polarity
subjetividade = exemplo.sentiment.subjectivity

df = pd.DataFrame([[frase, polaridade, subjetividade]], columns=['frase', 'polaridade', 'subjetividade'])
df

,frase,polaridade,subjetividade
0,"Gostei muito do filme, mas achei o final muito...",-0.033333,0.822222


## Pipeline

In [36]:
def traduz(frase):
    exemplo = TextBlob(frase)
    return exemplo.translate(from_lang='pt', to='en')

In [38]:
def trata(frase):
    text_tokens = word_tokenize(str(frase))
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('portuguese')]
    return (" ").join(tokens_without_sw)

In [39]:
def previsao(frase):
    exemplo = TextBlob(frase)
    polaridade = exemplo.sentiment[0]
    subjetividade = exemplo.sentiment[1]
    
    df = pd.DataFrame(
        [[frase, polaridade, subjetividade]],
        columns=["frase", "polaridade", "subjetividade"],
    )

    df["sentimento"] = "neutro"
    df.loc[df.polaridade < -0.25, "sentimento"] = "negativo"
    df.loc[df.polaridade > 0.25, "sentimento"] = "positivo"
    
    return df.to_json("dados.json", orient="columns"), polaridade, subjetividade

In [41]:
frase = input('Digite algo: ')
text = traduz(frase)
text = trata(text)
previsao(text)

(None, 0.7, 0.6000000000000001)